In [8]:
from find_source import make_catalog, combine_catalogs, summary
import glob
import pandas as pd

In [9]:
summary('../data/multi_track/v1195tau_11076.fits', True, False, False)

{'int_peak_val': 0.001923573319800198,
 'int_peak_coord': (0.0, -0.5509871812773864),
 'int_snr': 4.183568548840135,
 'calc_int_snr': 2.9433275024230423,
 'int_prob': 0.00033786365419613833,
 'calc_int_prob': 0.029803578791912533,
 'ext_peak_val': 'No significant external peak',
 'ext_peak_coord': 'No significant external peak',
 'ext_snr': 'No significant external peak',
 'calc_ext_snr': 2.8957778731563475,
 'ext_prob': 'No significant external peak',
 'calc_ext_prob': 1.0000000000000004,
 'field_center': (0, 0),
 'rms': 0.00045979247079230845,
 'calc_rms_val': 0.0006535369639350872,
 'n_incl_meas': 18.357342579134635,
 'n_excl_meas': 528.7927481691693,
 'radius': [15.216258970967269]}

In [10]:
final_catalog = {}
for file in glob.glob('../data/new_250609_17:30:22/*.fits'):
    try:
        final_catalog = combine_catalogs(final_catalog, make_catalog(file))
    except:
        print(f'Try again for {file}')

In [11]:
print(final_catalog)

{'source_1': {'field_name': '2232+117', 'obs_date_time': '6-10-25 11:16:38', 'file_name': '2232+117_full.fits', 'beam_maj_axis': <Angle 0.00047142 deg>, 'beam_min_axis': <Angle 0.00021899 deg>, 'beam_pos_angle': <Angle 99.00621 deg>, 'flux_uncertainty': 0.0251136664301157, 'flux_density': 3.2255163192749023, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (338.15170417, 11.73080667)>, 'internal': True}, 'source_2': {'field_name': '2232+117', 'obs_date_time': '6-10-25 11:16:38', 'file_name': '2232+117_full.fits', 'beam_maj_axis': <Angle 0.00047142 deg>, 'beam_min_axis': <Angle 0.00021899 deg>, 'beam_pos_angle': <Angle 99.00621 deg>, 'flux_uncertainty': 0.0251136664301157, 'flux_density': 0.17307700216770172, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (338.15579356, 11.72991101)>, 'internal': False}, 'source_3': {'field_name': '2232+117', 'obs_date_time': '6-10-25 11:16:38', 'file_name': '2232+117_full.fits', 'beam_maj_axis': <Angle 0.00047142 deg>, 'beam_min_axis': <Angle 0.0002189

In [12]:
df = pd.DataFrame.from_dict(final_catalog)
print(df)

                                                           source_1  \
field_name                                                 2232+117   
obs_date_time                                      6-10-25 11:16:38   
file_name                                        2232+117_full.fits   
beam_maj_axis                                     0d00m01.69712316s   
beam_min_axis                                     0d00m00.78837516s   
beam_pos_angle                                        99d00m22.356s   
flux_uncertainty                                           0.025114   
flux_density                                               3.225516   
coord             <SkyCoord (ICRS): (ra, dec) in deg\n    (338.1...   
internal                                                       True   

                                                           source_2  \
field_name                                                 2232+117   
obs_date_time                                      6-10-25 11:16:38   
file_